<a href="https://colab.research.google.com/github/abhishekydv14/Sales_Analysis_Project/blob/main/Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
file_path = '/content/drive/MyDrive/My Data Analytics Project/superstore.csv'
df = pd.read_csv(file_path)
# --- DATA CLEANING ---
df['Order Date'] = pd.to_datetime(df['Order.Date'])
print("Data has been loaded and cleaned successfully!")
df.info()

Data has been loaded and cleaned successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Category        51290 non-null  object        
 1   City            51290 non-null  object        
 2   Country         51290 non-null  object        
 3   Customer.ID     51290 non-null  object        
 4   Customer.Name   51290 non-null  object        
 5   Discount        51290 non-null  float64       
 6   Market          51290 non-null  object        
 7   记录数             51290 non-null  int64         
 8   Order.Date      51290 non-null  object        
 9   Order.ID        51290 non-null  object        
 10  Order.Priority  51290 non-null  object        
 11  Product.ID      51290 non-null  object        
 12  Product.Name    51290 non-null  object        
 13  Profit          51290 non-null  float64       
 14  Quantit

In [4]:
# --- ANALYSIS 1: PRODUCT PROFITABILITY ---
profit_by_product = df.groupby('Product.Name')['Profit'].sum()

top_15_products = profit_by_product.nlargest(15)

# 3. Print the result.
print("--- Top 15 Most Profitable Products ---")
print(top_15_products)

--- Top 15 Most Profitable Products ---
Product.Name
Canon imageCLASS 2200 Advanced Copier                                          25199.92800
Cisco Smart Phone, Full Size                                                   17238.52060
Motorola Smart Phone, Full Size                                                17027.11300
Hoover Stove, Red                                                              11807.96900
Sauder Classic Bookcase, Traditional                                           10672.07300
Harbour Creations Executive Leather Armchair, Adjustable                       10427.32600
Nokia Smart Phone, Full Size                                                    9938.19550
Cisco Smart Phone, with Caller ID                                               9786.64080
Nokia Smart Phone, with Caller ID                                               9465.32570
Belkin Router, USB                                                              8955.01800
Hewlett Wireless Fax, High-Speed     

In [5]:
# VERIFY THE KEY INSIGHT ---

profit_by_product_sorted = df.groupby('Product.Name')['Profit'].sum().sort_values(ascending=False)


total_profit = df['Profit'].sum()

#  top 15%
num_products = len(profit_by_product_sorted)
top_15_percent_count = int(num_products * 0.15)


profit_from_top_15_percent = profit_by_product_sorted.head(top_15_percent_count).sum()

percentage_from_top_15 = (profit_from_top_15_percent / total_profit) * 100

# Print the final, correct results
print(f"Percentage of total profit from top 15% of products: {percentage_from_top_15:.2f}%")

Percentage of total profit from top 15% of products: 84.61%


In [6]:
# --- ANALYSIS 3: REGIONAL PERFORMANCE ---

regional_performance = df.groupby('Region')[['Sales', 'Profit']].sum()

# Calculate the Profit Margin (Profit / Sales) for each region.

regional_performance['Profit Margin'] = (regional_performance['Profit'] / regional_performance['Sales']) * 100

regional_performance = regional_performance.sort_values(by='Profit Margin', ascending=False)

print("Regional Performance (Sorted by Profit Margin) -")
print(regional_performance)

Regional Performance (Sorted by Profit Margin) -
                  Sales        Profit  Profit Margin
Region                                              
Canada            66932   17817.39000      26.620137
North Asia       848349  165578.42100      19.517725
Central Asia     752839  132480.18700      17.597413
North           1248192  194597.95252      15.590386
West             725514  108418.44890      14.943674
East             678834   91522.78000      13.482351
Africa           783776   88871.63100      11.338907
Central         2822399  311403.98164      11.033308
Oceania         1100207  120089.11200      10.915138
Caribbean        324281   34571.32104      10.660915
South           1600960  140355.76618       8.766975
EMEA             806184   43897.97100       5.445155
Southeast Asia   884438   17852.32900       2.018494


In [8]:
import sqlite3
conn = sqlite3.connect('sales_database.db')

df.to_sql('sales', conn, if_exists='replace', index=False)

sql_query = """
SELECT
    Region,
    SUM(Sales) AS TotalSales,
    SUM(Profit) AS TotalProfit,
    (SUM(Profit) / SUM(Sales)) * 100 AS ProfitMargin
FROM
    sales
GROUP BY
    Region
ORDER BY
    ProfitMargin DESC;
"""

sql_results = pd.read_sql_query(sql_query, conn)
print("\n--- Regional Performance Results using SQL ---")
print(sql_results)


conn.close()


--- Regional Performance Results using SQL ---
            Region  TotalSales   TotalProfit  ProfitMargin
0           Canada       66932   17817.39000     26.620137
1       North Asia      848349  165578.42100     19.517725
2     Central Asia      752839  132480.18700     17.597413
3            North     1248192  194597.95252     15.590386
4             West      725514  108418.44890     14.943674
5             East      678834   91522.78000     13.482351
6           Africa      783776   88871.63100     11.338907
7          Central     2822399  311403.98164     11.033308
8          Oceania     1100207  120089.11200     10.915138
9        Caribbean      324281   34571.32104     10.660915
10           South     1600960  140355.76618      8.766975
11            EMEA      806184   43897.97100      5.445155
12  Southeast Asia      884438   17852.32900      2.018494


In [9]:
# --- SAVE THE CLEANED DATA TO GOOGLE DRIVE ---
save_path = '/content/drive/MyDrive/My Data Analytics Project/cleaned_superstore_data.csv'

df.to_csv(save_path, index=False)

print(f"Success! cleaned data has been saved to your Google Drive.")
print(f"now find it at this path: {save_path}")

Success! cleaned data has been saved to your Google Drive.
now find it at this path: /content/drive/MyDrive/My Data Analytics Project/cleaned_superstore_data.csv
